#0.&nbsp; Imports und Helper

In [2]:
import torch
import numpy as np
from torch import nn
from math import pi, e
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from scipy.interpolate import griddata
import plotly.graph_objects as go
from sklearn.datasets import load_iris, load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch.nn.functional as F
from torch.autograd import Variable
import tqdm
import torch.nn.init as init



# 1.&nbsp;Gradient Implementieren

## Analytisch


Gegeben sei die folgende Funktion:

\begin{equation}
h(a, b) = 3\cdot(1-a)^2-b^2
\end{equation}

Bestimmen Sie den Gradientenvektor unter Verwendung von Schulmathematik (Ableitungsregeln):

\begin{equation}
  \begin{bmatrix}
  \dfrac{\partial f}{\partial a} ,\dfrac{\partial f}{\partial b}
  \end{bmatrix}
\end{equation}



**Kettenregel**: Für eine zusammengesetzte Funktion gilt $h(x) = f(g(x)) \equiv (f \circ g)(x)$:

\begin{equation}
f'(x) = g'(h(x)) \cdot h'(x)
\end{equation}

oder:

\begin{equation}
\frac{\partial f(g(x))}{\partial x} = \frac{\partial f(g(x))}{\partial g(x)} ~ \frac{\partial g(x)}{\partial x}
\end{equation}


h'(a) = -6(1-a)  
h'(b) = -2b

## Programmatisch



*   Implementieren Sie Pythonfunktionen, die Ihnen jeweils die Funktionswerte und die Werte der partiellen Ableitungen liefern.
* Nutzen Sie die vorgegebene Codestruktur, um zunächst die Funktionswerte als 3d-Oberflächenplot zu visualisieren
*   Fügen Sie weiterhin einen beliebigen Startpunkt zu Visualisierung hinzu und weiterhin die nächste Position nach Durchführung eines Schrittes im Gradientenabstieg. Sieht das Ergebnis visuell plausibel aus?
* Experimentieren Sie mit der Lernrate und dem Ergebnis z.B. nach mehreren Iterationen



In [4]:
def forward(a,b):
  return 3*(1-a)**2 - b**2

# from sympy import Symbol
# def backward(x:str, a, b):
#   h = 3*(1-a)**2 - b**2
#   return h.diff(Symbol(x))

def partial_derivate_a(a):
  return -6*(1-a)

def partial_derivate_b(b):
  return -2*b

def backward(a,b,alpha):
  # alpha == Lernrate
  a_new = a - alpha * partial_derivate_a(a)
  b_new = b - alpha * partial_derivate_b(b)
  return a_new, b_new

In [20]:
a=-2.5
b=0.8
stepsize = 0.1 # 0.01

smallestpoint = -3
largestpoint = 3

x, y = np.mgrid[smallestpoint:largestpoint:stepsize, smallestpoint:largestpoint:stepsize]

# Fläche zeichnen
fig = go.Figure([
    go.Surface(x=x, y=y, z=forward(x,y))
])

a_neu, b_neu = backward(a,b,stepsize)
a_zwei, b_zwei = backward(a_neu, b_neu, stepsize) # 2. Iteration
a_drei, b_drei = backward(a_zwei, b_zwei, stepsize) # 3. Iteration

# Einzelne Punkte ergänzen
fig.add_trace(
    go.Scatter3d(x=[a, a_neu, a_zwei, a_drei],
                 y=[b, b_neu, b_zwei, a_drei],
                 z=[forward(a,b), forward(a_neu, b_neu), forward(a_zwei, b_zwei), forward(a_drei, b_drei)],
                 mode='markers',
                 hovertext=['(Input)', '(1. Iteration)', '(2. Iteration)', '(3. Iteration)'])
)

fig.update_layout(
    title=f'Startwert a: {a}, Startwert b: {b}, Lernrate: {stepsize}'
)

fig

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'type': 'surface',
              'x': array([[-3. , -3. , -3. , ..., -3. , -3. , -3. ],
                          [-2.9, -2.9, -2.9, ..., -2.9, -2.9, -2.9],
                          [-2.8, -2.8, -2.8, ..., -2.8, -2.8, -2.8],
                          ...,
                          [ 2.7,  2.7,  2.7, ...,  2.7,  2.7,  2.7],
                          [ 2.8,  2.8,  2.8, ...,  2.8,  2.8,  2.8],
                          [ 2.9,  2.9,  2.9, ...,  2.9,  2.9,  2.9]]),
              'y': array([[-3. , -2.9, -2.8, ...,  2.7,  2.8,  2.9],
                          [-3. , -2.9, -2.8, ...,  2.7,  2.8,  2.9],
                          [-3. , -2.9, -2.8, ...,  2.7,  2.8,  2.9],
                          ...,
                          [-3. , -2.9, -2.8, ...,  2.7,  2.8,  2.9],
                          [-3. , -2.9, -2.8, ...,  2.7,  2.8,  2.9],
                          [-3. , -2.9, -2.8, ...,  2.7,  2.8,  2.9]]),
              'z': array([[39.  , 39.59, 40.16, ..., 40.71, 40.16, 39.59],
                          [36.63, 37.22, 37.79, ..., 38.34, 37.79, 37.22],
                          [34.32, 34.91, 35.48, ..., 36.03, 35.48, 34.91],
                          ...,
                          [-0.33,  0.26,  0.83, ...,  1.38,  0.83,  0.26],
                          [ 0.72,  1.31,  1.88, ...,  2.43,  1.88,  1.31],
                          [ 1.83,  2.42,  2.99, ...,  3.54,  2.99,  2.42]])},
             {'hovertext': [(Input), (1. Iteration), (2. Iteration), (3.
                            Iteration)],
              'mode': 'markers',
              'type': 'scatter3d',
              'x': [-2.5, -0.3999999999999999, 0.43999999999999995, 0.776],
              'y': [0.8, 0.9600000000000001, 1.1520000000000001, 0.776],
              'z': [36.11, 4.958399999999999, -0.3863040000000002,
                    -1.7605017600000001]}],
    'layout': {'template': '...', 'title': {'text': 'Startwert a: -2.5, Startwert b: 0.8, Lernrate: 0.1'}}
})

## PyTorch Autograd

Zeigen bzw. überprüfen Sie mit Hilfe der in PyTorch vorhandenen autograd Funktionalität (z.B. https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html die Korrektheit Ihrer oben "zu Fuß" implementiert berechneten Ergebnisse. Greifen Sie ggf. auch auf die separat referenzierten und Tutorials zurück.

In [6]:
import torch

a_tens = torch.tensor([a], requires_grad=True)
b_tens = torch.tensor([b], requires_grad=True)

h = forward(a_tens,b_tens)

h.backward()

In [7]:
print('\t Autograd\t\t Per Hand')
print('a:\t', a_tens.grad,'\t', partial_derivate_a(a))
print('b:\t', b_tens.grad,'\t', partial_derivate_b(b))

	 Autograd		 Per Hand
a:	 tensor([-21.]) 	 -21.0
b:	 tensor([-1.6000]) 	 -1.6
